In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from skopt import BayesSearchCV

from skopt.space import Real, Categorical, Integer
import pandas as pd
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import StratifiedKFold

In [6]:
train_data_loaded = pd.read_csv('../data/train_data_2024-08-01.csv')

X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KFold-Konfiguration
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definieren des F-beta-Scores mit beta = 2
beta = 2
fbeta_scorer = make_scorer(fbeta_score, beta=beta)

In [7]:
# Parameter-Suchräume definieren
#class_weights = ['balanced', {0: 1, 1: 2, 2: 1}, {0: 1, 1: 1, 2: 2}, None]
#class_weights = ['balanced', {0:1, 1: 1}, {0:1, 1: 3}, {0:1, 1: 6}, {0:1, 1: 7}, {0:1, 1: 8}, {0:1, 1: 9}, {0:1, 1: 10}, None]
rf_param_space = {
    'n_estimators': list(range(100, 1500, 10)),
    'max_depth': list(range(5, 30)),
    'min_samples_split': list(range(2, 200)),
    'min_samples_leaf': list(range(1, 20)),
    'max_features': ['sqrt', 'log2',2, 4, 6, 8, 10, 12, 14, 16, 18],
   
    
}

In [8]:
# Bayessche Optimierung durchführen

# Random Forest
rf_opt = BayesSearchCV(estimator=BalancedRandomForestClassifier(), search_spaces=rf_param_space, n_iter=32, cv=kf, n_jobs=-1, scoring=fbeta_scorer, random_state=42)
rf_opt.fit(X_train, y_train)
print("Best parameters for Random Forest: ", rf_opt.best_params_)

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this w

Best parameters for Random Forest:  OrderedDict({'max_depth': 20, 'max_features': 14, 'min_samples_leaf': 7, 'min_samples_split': 175, 'n_estimators': 920})


In [12]:
# Modell evaluieren
# Random Forest
rf_best = rf_opt.best_estimator_
rf_pred = rf_best.predict(X_test)
scores = cross_val_score(rf_best, X, y, cv=5, scoring=fbeta_scorer)
print(f'F-Beta-Score für jede Falte: {scores}')
print(f'Durchschnittlicher F-Beta-Score: {scores.mean()}')


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

F-Beta-Score für jede Falte: [0.46071473 0.45592705 0.46970081 0.48778984 0.47098298]
Durchschnittlicher F-Beta-Score: 0.4690230811339918


In [13]:
print("Best fbeta_score for Balanced Random Forest: ", rf_opt.best_score_)


Best fbeta_score for Balanced Random Forest:  0.466824549700525
